In [20]:
# imports + load scouting data

import os
import pandas as pd
import numpy as np
import requests

from data_container import load_scouted_data  # and load_pit_data if you need it

df = load_scouted_data().copy()
pd.set_option('display.max_columns', None)

# columns
TEAM_COL  = "Team Number" if "Team Number" in df.columns else ("Team" if "Team" in df.columns else None)
MATCH_COL = "Match Number" if "Match Number" in df.columns else ("Match" if "Match" in df.columns else None)

if TEAM_COL is None:
    raise ValueError("Need a team column: 'Team Number' or 'Team'.")
if MATCH_COL is None:
    raise ValueError("Need a match column: 'Match Number' or 'Match'.")

# numeric fuel
for c in ["Auto Fuel", "Teleop Fuel"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    else:
        df[c] = 0

df["Total Fuel in Hub"] = df["Auto Fuel"] + df["Teleop Fuel"]

In [21]:
# pull data from TBA

TBA_KEY = "PUT_YOUR_TBA_KEY_HERE"
EVENT_KEY = "2026gpr"  #should change per comp

headers = {"X-TBA-Auth-Key": TBA_KEY}
url = f"https://www.thebluealliance.com/api/v3/event/{EVENT_KEY}/matches/simple"
matches = requests.get(url, headers=headers).json()

# (Match Number, Team Number) -> Alliance + Match ID

lookup_rows = []
for m in matches:
    comp = m["comp_level"]          # qm / qf / sf / f
    match_num = m["match_number"]
    set_num = m.get("set_number", 1)

    # match id
    match_id = f"{comp.upper()}{match_num}"
    if comp != "qm":
        match_id = f"{comp.upper()}-{set_num}-{match_num}"

    for t in m["alliances"]["red"]["team_keys"]:
        lookup_rows.append({
            "Match ID": match_id,
            "Comp Level": comp,
            "Match Number": match_num,
            "Team Number": int(t.replace("frc", "")),
            "Alliance": "Red"
        })

    for t in m["alliances"]["blue"]["team_keys"]:
        lookup_rows.append({
            "Match ID": match_id,
            "Comp Level": comp,
            "Match Number": match_num,
            "Team Number": int(t.replace("frc", "")),
            "Alliance": "Blue"
        })

tba_lookup = pd.DataFrame(lookup_rows)

# merge into scouting data
df = df.merge(
    tba_lookup[["Match ID", "Match Number", "Team Number", "Alliance", "Comp Level"]],
    left_on=[MATCH_COL, TEAM_COL],
    right_on=["Match Number", "Team Number"],
    how="left"
)

# set ALLI_COL
ALLI_COL = "Alliance"

# normalize alliance labels
df[ALLI_COL] = df[ALLI_COL].astype(str).str.strip().str.title()
df.loc[df[ALLI_COL].isin(["R", "Red Alliance"]), ALLI_COL] = "Red"
df.loc[df[ALLI_COL].isin(["B", "Blue Alliance"]), ALLI_COL] = "Blue"

TypeError: string indices must be integers, not 'str'

In [5]:
# avg fuel scored per alliance

alliance_stats = (
    df.groupby(["Match ID", ALLI_COL])
      .agg(
          alliance_total_fuel=("Total Fuel in Hub", "sum"),
          alliance_avg_fuel=("Total Fuel in Hub", "mean"),
          teams_count=(TEAM_COL, "nunique"),
      )
      .reset_index()
      .sort_values(["Match ID", ALLI_COL])
)


KeyError: 'Match ID'

In [6]:
# predict r vs b 

pivot = alliance_stats.pivot(
    index="Match ID",
    columns=ALLI_COL,
    values="alliance_total_fuel"
).reset_index()

if "Red" not in pivot.columns:
    pivot["Red"] = 0
if "Blue" not in pivot.columns:
    pivot["Blue"] = 0

pivot["Fuel_Diff"] = pivot["Red"] - pivot["Blue"]
pivot["Predicted_Winner"] = np.where(pivot["Fuel_Diff"] >= 0, "Red", "Blue")

NameError: name 'alliance_stats' is not defined

In [7]:
#export to csv

radio_df = pivot[["Match ID", "Red", "Blue", "Fuel_Diff", "Predicted_Winner"]].sort_values("Match ID")
radio_df.to_csv("match_predictions_all_matches.csv", index=False)

alliance_stats.head(10), radio_df.head(10)

NameError: name 'pivot' is not defined